In [ ]:
# ---------------------------
# 1. התקנת חבילות (רק פעם ראשונה)
# ---------------------------
# pip install ultralytics opencv-python torch torchvision numpy filterpy

# אם רוצים OC-SORT:
# pip install motmetrics

# ---------------------------
# 2. קוד Python
# ---------------------------
import cv2
from ultralytics import YOLO
import numpy as np

# ---------------------------
# YOLOv8 - Object Detection
# ---------------------------
model = YOLO("yolov8n.pt")  # Nano = מהיר, טוב לוידאו בזמן אמת

# ---------------------------
# פתיחת מצלמה
# ---------------------------
cap = cv2.VideoCapture(0)  # 0 = מצלמת המחשב, או נתיב לוידאו

# ---------------------------
# יצירת tracker בסיסי
# ---------------------------
class SimpleTracker:
    def __init__(self):
        self.next_id = 0
        self.objects = {}  # id -> bbox

    def update(self, detections):
        new_objects = {}
        for det in detections:
            # כל object מקבל ID חדש (פשוט, לא מתוחכם כמו OC-SORT)
            new_objects[self.next_id] = det
            self.next_id += 1
        self.objects = new_objects
        return self.objects

tracker = SimpleTracker()

# ---------------------------
# 3. לולאת אינפרנס
# ---------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]  # detection
    boxes = results.boxes.xyxy.cpu().numpy()  # [N,4]
    scores = results.boxes.conf.cpu().numpy() # confidence
    labels = results.boxes.cls.cpu().numpy()  # class index

    # מסנן לפי סף אמון
    mask = scores > 0.3
    boxes = boxes[mask]
    labels = labels[mask]

    # עדכון tracker (פשוט)
    tracked_objects = tracker.update(boxes)

    # ציור תיבות
    for obj_id, bbox in tracked_objects.items():
        xmin, ymin, xmax, ymax = map(int, bbox)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(frame, f"ID:{obj_id}", (xmin, ymin-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    cv2.imshow("Realtime Detection + Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
